Import of necessary libraries

In [11]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split



Dataset clean (only once)

In [9]:
dataset= pd.read_csv("explaination_toxic_conversation_most_toxic_sentences.csv",header=0, index_col=0, encoding='utf-8')
dataset = dataset[dataset['explaination'].str.contains("La frase più tossica è", na=False)]
#add a new column 'toxic' with value 1 for all rows
dataset['toxic'] = 1

# View the first few rows of the dataset with the new 'toxic_phrase' column

non_toxic_dataset= pd.read_csv("generated_dataset.csv",header=0, index_col=0, encoding='utf-8')
#concatenate the two datasets
dataset = pd.concat([dataset, non_toxic_dataset], ignore_index=False)
dataset.to_csv("filtered_dataset.csv", index=True, encoding='utf-8')

                                   toxic
person_couple                           
Psicopatico e Adulatrice               1
Manipolatore e Dipendente emotiva      1
Persona violenta e Succube             1
Psicopatico e Adulatrice               1
Narcisista e Succube                   1


SET preparation

In [21]:

dataset=pd.read_csv("filtered_dataset.csv",header=0, index_col=0, encoding='utf-8')
#set seed for reproducibility
seed = 42
sklearn.utils.check_random_state(seed)
#shuffle the dataset
dataset = dataset.sample(frac=1, random_state=seed)
#take 70% of the dataset for training and 30% for testing, toxic column is the target
train_size = int(0.7 * len(dataset))
X_train, X_test, y_train, y_test = train_test_split(
    dataset['conversation'],  # Solo la colonna conversation
    dataset['toxic'],
    train_size=train_size,
    random_state=seed,
    shuffle=True
)

#print the shape of every set
print(f"Training set shape: {X_train.shape}, Test set shape: {X_test.shape}")
print(f"Training labels shape: {y_train.shape}, Test labels shape: {y_test.shape}")

Training set shape: (1032,), Test set shape: (443,)
Training labels shape: (1032,), Test labels shape: (443,)
